When we start up the server it first creates a raw 'socket' and then 'binds' the socket to an end point.  The end point is in reality and internet facing buffer inside the kernel capable of receiving data from a client.  The server then issues a 'listen' call which converts the raw socket into a listening socket.

In [1]:
import socket, sys
from threading import Thread

PORT = 7002
listenSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
listenSocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
listenSocket.bind(('', PORT))
listenSocket.listen(5)    # set up backlog buffer
messageNo = 1

Next the server performs an 'accept' call.  This is a blocking call and will not return until a client connects.

The client is then started and attempts to connect to the server.

In [2]:
print(f"Server starting on port {PORT}")

# block until we accept a client connection and then create a second socket
newsocket, remoteIPandPORT = listenSocket.accept()
print("SERVER: opened a new connection:", remoteIPandPORT)

Server starting on port 7002
SERVER: opened a new connection: ('127.0.0.1', 54846)


After the client successfully connects, the server will return from the blocking 'accept' call and creates a second socket for transfering data between the server and the client.  The listening socket is retained to allow further clients to connect.  The server can now either send and receive data with the client or accept further connections.  

To allow the server to both it is normal to create a new server thread at this point to communicate with the first client.  The original thread continues to wait for further client connections.  The new thread will execute the following code:

In [3]:
def communicateWithClient(newsocket, messageNo):
    # wait for message and echo
    message = newsocket.recv(100)
    print("SERVER:", message.decode("UTF-8"))
    sys.stdout.flush()
    
    # send response and close socket immediately
    response = "message {0} - {1}".format(messageNo, message.decode("UTF-8"))
    newsocket.send(response.encode("UTF-8"))
    print(f"closing {newsocket}")
    newsocket.close()


The server thread is now started.

In [5]:
clientThread = Thread(target=communicateWithClient, args=(newsocket, messageNo))
messageNo = messageNo + 1
clientThread.start()

SERVER: This is a message from a client.
closing <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 7002), raddr=('127.0.0.1', 54846)>
